In [1]:
%matplotlib inline

from numpythia import Pythia, STATUS, HAS_END_VERTEX, ABS_PDG_ID
from pyjet import cluster
import matplotlib.pyplot as plt
import numpy as np
import ROOT
from ROOT import gRandom, TH1, TH1D, cout, TCanvas
ROOT.gStyle.SetOptStat('')


Welcome to JupyROOT 6.15/01


In [2]:
selection = ((STATUS == 1) & ~HAS_END_VERTEX &
 (ABS_PDG_ID != 12) & (ABS_PDG_ID != 14) & (ABS_PDG_ID != 16))  #removing neutrinos

In [8]:
params = {
 "Beams:frameType":  2,   
 "Beams:idA": 2212, 
 "Beams:idB": 11,
 "Beams:eA" : 250,
 "Beams:eB"  : 20,
 "WeakBosonExchange:ff2ff(t:gmZ)": "on",
 "PhaseSpace:Q2Min": 10,
 #"SpaceShower:dipoleRecoil": "on",
 "SpaceShower:pTmaxMatch": 2,
 "PDF:lepton": "off",
 "TimeShower:QEDshowerByL": "off",
}

In [3]:
##PHOTOPRODUCTION

In [ ]:
params = {
 "HardQCD:all": "on",
 "Beams:eCM": 100,
 "Beams:idA": 11,
 "Beams:idB": 2212,
 "PDF:lepton2gamma": "on",
 "Photon:Q2max":1.0, 
 #"Photon:Wmin":10.0,
 #"PhaseSpace:pTHatMin":5.0,
 "PhotonParton:all": "on",
 #"Photon:ProcessType":0,
 "Main:timesAllowErrors":10000,
}


In [ ]:

#    // 4 = direct-direct.
#np.dtype([('E', 'f8'), ('px', 'f8'), ('py', 'f8'), ('pz', 'f8'), ('mass', 'f8'),
#          ('prodx', 'f8'), ('prody', 'f8'), ('prodz', 'f8'), ('prodt', 'f8'),
#          ('pdgid', 'i4'), ('status', 'i4')])

In [9]:
pythia = Pythia(params=params)


 *-------  PYTHIA Process Initialization  --------------------------*
 |                                                                  |
 | We collide p+ with e- at a CM energy of 1.414e+02 GeV            |
 |                                                                  |
 |------------------------------------------------------------------|
 |                                                    |             |
 | Subprocess                                    Code |   Estimated |
 |                                                    |    max (mb) |
 |                                                    |             |
 |------------------------------------------------------------------|
 |                                                    |             |
 | f f' -> f f' (t-channel gamma*/Z0)             211 |   1.896e-04 |
 |                                                                  |
 *-------  End PYTHIA Process Initialization -----------------------*

 *-------  PYTHIA 

In [24]:
hpt = ROOT.TH1F("hpt","hpt",20,0,10.0)
heta = ROOT.TH1F("heta","heta",20,-8.0,8.0)
h_nch = ROOT.TH1F("h_nch","h_nch",20,0,20)
h_jetpt = ROOT.TH1F("h_jetpt","h_jetpt", 20, 0, 20)
h_jetmass = ROOT.TH1F("h_jetmass", "h_jetmass", 20,0,20)
hpt.Sumw2()
heta.Sumw2()
h_nch.Sumw2()
h_jetpt.Sumw2()
h_jetmass.Sumw2()

Warning in <TROOT::Append>: Replacing existing TH1: hpt (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: heta (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_nch (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_jetpt (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_jetmass (Potential memory leak).


In [92]:
for event in pythia(events=10):
    if not event: continue
        
        
    #need to find proton, and incoming and outgoing electron to get the DIS variables: Q2, W, Xbj, y
    print 'proton'
    print event.all()[0]
    
    
    #incoming and outgoing electron:
    print 'electron'
    for i in event.all():
        if i['pdgid']==11 and (i['status']==4 or i['status']==1):
            print i

    
    jets = cluster(event.all(selection &(ABS_PDG_ID != 11)), R=1, p=-1, ep=True).inclusive_jets(ptmin=5)
    print('This events has # ', len(jets), 'jets')
    for jet in jets:
        h_jetpt.Fill(jet.pt)
        h_jetmass.Fill(jet.mass)
        print ' jet' , jet.pt, ' ' , jet.eta,  ' ' , jet.phi, ' mass', jet.mass, ' #constituents: ' , len(jet)
        #look for constituents in jets: 
        #for constit in jet:
            
            #print constit
            
        
    nch = 0
    for p in event.all(selection & (ABS_PDG_ID == 211)):
        pt = np.sqrt(p['px']*p['px']* + p['py']*p['py'])
        p3 = np.sqrt(p['px']*p['px']* + p['py']*p['py'] + p['pz']*p['pz'])
        eta = 0.5*np.log( (p3 + p['pz'])/(p3-p['pz']))
        if(pt<0.050): continue
        #print p['E'], ' ', p['pdgid'], ' ', pt
        hpt.Fill(pt) 
        heta.Fill(eta)
        nch +=1 
    h_nch.Fill(nch)
#print ' '
    

proton
(250., 2.22044605e-16, 4.4408921e-16, 249.99823929, 0.93827, 0., 0., 0., 0., 2212, 4)
electron
(20., -2.22044605e-16, 0., -19.99999999, 0.000511, 0., 0., 0., 0., 11, 4)
(10.26808737, -1.43523778, -2.46895216, -9.86296029, 0.00051097, 0., 0., 0., 0., 11, 1)
('This events has # ', 1, 'jets')
 jet 6.59815785137   0.618129213858   1.98219569182  mass 3.16508107255  #constituents:  5
proton
(250., -4.48530102e-14, 1.88737914e-14, 249.99823929, 0.93827, 0., 0., 0., 0., 2212, 4)
electron
(20., 5.68434189e-14, -7.10542736e-14, -19.99999999, 0.0005111, 0., 0., 0., 0., 11, 4)
(19.98423383, -2.5839295, 2.14356153, -19.70020441, 0.00051142, 0., 0., 0., 0., 11, 1)
('This events has # ', 0, 'jets')
proton
(250., 0., 7.77156117e-16, 249.99823929, 0.93827, 0., 0., 0., 0., 2212, 4)
electron
(20., 0., 1.33226763e-15, -19.99999999, 0.00051101, 0., 0., 0., 0., 11, 4)
(11.55215665, -2.32059343, -1.81162829, -11.17072836, 0.000511, 0., 0., 0., 0., 11, 1)
('This events has # ', 0, 'jets')
proton
(250.

In [26]:
c = ROOT.TCanvas('c','c',800,800)
h_nch.Draw()
c.Draw()
c.SaveAs("h_nch.pdf")
c.Clear()
heta.Draw()
c.SaveAs("h_eta.pdf")
c.Clear()
hpt.Draw()
ROOT.gPad.SetLogy()
c.SaveAs("h_pt.pdf")
c.Clear()
h_jetpt.Draw()
c.SaveAs("h_jetpt.pdf")
c.Clear()
h_jetmass.Draw()
c.SaveAs("h_jetmass.pdf")


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: pdf file h_nch.pdf has been created
Info in <TCanvas::Print>: pdf file h_eta.pdf has been created
Info in <TCanvas::Print>: pdf file h_pt.pdf has been created
Info in <TCanvas::Print>: pdf file h_jetpt.pdf has been created
Info in <TCanvas::Print>: pdf file h_jetmass.pdf has been created
